In [1]:
import numpy as np
import pandas as pd
import time
from datetime import date, datetime
import os

os.environ["config_path"] = "/home/asingal/home/SeQUeNCe/example/trajectree_integration/MIM_config.json"

from sequence.topology.router_net_topo import RouterNetTopo
from sequence.app.request_app import RequestApp
import sequence.utils.log as log


/home/asingal/home/.venv/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/asingal/home/.venv/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


Using Trajectree backend for entanglement generation


In [2]:
today = date.today()

CONFIG_FILE = "MIM_config.json"

# meta params
NO_TRIALS = 10
OUTPUT_FILE = "results/var_mem_"+str(today)+".csv"
LOGGING = False
LOG_OUTPUT = "results/var_mem_log.csv"
MODULE_TO_LOG = ["timeline", "memory", "bsm", "generation", "request_app"]

# simulation params
PREP_TIME = int(1e12)  # 1 second
COLLECT_TIME = int(3e12)  # 10 seconds


# qc params
# QC_FREQ = 1e11
QC_FREQ = 1e11



# application params
APP_NODE_NAME = "left"
OTHER_NODE_NAME = "right"
num_memories = [4,8,16,32,64]  # iterate through these


# storing data
data_dict = {"Memory Count": [],
             "Average Throughput": [],
             "Std. Throughput": [],
             "Average TTT": [],
             "Std. TTT":[],
             }


tick = time.time()

collect_time_base = 3e12

for i, num_memo in enumerate(num_memories):
    print(f"Running {NO_TRIALS} trials for memory count {num_memo} ({i + 1}/{len(num_memories)})")
    data_dict["Memory Count"].append(num_memo)
    throughputs = np.zeros(NO_TRIALS)
    time_to_thousand = np.zeros(NO_TRIALS)

    COLLECT_TIME = collect_time_base/num_memo

    for trial_no in range(NO_TRIALS):
        # establish network
        net_topo = RouterNetTopo(CONFIG_FILE)

        # timeline setup
        tl = net_topo.get_timeline()
        tl.stop_time = PREP_TIME + COLLECT_TIME

        if LOGGING:
            # set log
            if num_memo == num_memories[0]:
                log.set_logger(__name__, tl, LOG_OUTPUT)
                log.set_logger_level('WARN')
                for module in MODULE_TO_LOG:
                    log.track_module(module)
            elif num_memo == num_memories[1]:
                for module in MODULE_TO_LOG:
                    log.remove_module(module)

        # network configuration
        routers = net_topo.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER)
        bsm_nodes = net_topo.get_nodes_by_type(RouterNetTopo.BSM_NODE)

        for j, node in enumerate(routers + bsm_nodes):
            node.set_seed(int(datetime.now().timestamp()) + j + (trial_no * 3))

        # set quantum channel parameters
        for qc in net_topo.get_qchannels():
            qc.frequency = QC_FREQ

        # establish app on left node
        start_node = None
        for node in routers:
            if node.name == APP_NODE_NAME:
                start_node = node
                break
        if not start_node:
            raise ValueError(f"Invalid app node name {APP_NODE_NAME}")
        end_node = None
        for node in routers:
            if node.name == OTHER_NODE_NAME:
                end_node = node
                break
        if not start_node:
            raise ValueError(f"Invalid other node name {OTHER_NODE_NAME}")

        app_start = RequestApp(start_node)
        app_end = RequestApp(end_node)

        # initialize and start app
        tl.init()
        app_start.start(OTHER_NODE_NAME, PREP_TIME, PREP_TIME + COLLECT_TIME, num_memo, 1.0)
        tl.run()

        # Used for debugging
        # attempt = app_start.node.total_attempts
        # success = app_start.node.succesful_attempts
        # prob = success/attempt

        throughputs[trial_no] = app_start.get_throughput()        
        

        print(f"\tCompleted trial {trial_no + 1}/{NO_TRIALS}")



    print("Finished trials.")

    avg_throughput = np.mean(throughputs)
    std_throughput = np.std(throughputs)
    avg_TTT = np.mean(time_to_thousand)*1e-12
    std_TTT = np.std(time_to_thousand)*1e-12
    
    print(f"Average throughput: {avg_throughput} +/- {std_throughput}")
    print(f"time to thousand entanglements:", time_to_thousand)

    data_dict["Average Throughput"].append(avg_throughput)
    data_dict["Std. Throughput"].append(std_throughput)
    data_dict["Average TTT"].append(avg_TTT)
    data_dict["Std. TTT"].append(std_TTT)
             

df = pd.DataFrame(data_dict)
df.to_csv(OUTPUT_FILE, index=False)

runtime = time.time() - tick
print("Total runtime:", runtime)


Running 10 trials for memory count 4 (1/5)
calling trajectree ent generation
checking for cached entanglement
cached entanglement not found, running simulation
completed set cache_hits: 509 cache_partial_hits: 0 cache_misses: 3 time taken: 38.71548247337341
calling trajectree ent generation
checking for cached entanglement
cached entanglement found: (0.9883998311890938, 0.3939867684050529)
calling trajectree ent generation
checking for cached entanglement
cached entanglement found: (0.9883998311890938, 0.3939867684050529)
calling trajectree ent generation
checking for cached entanglement
cached entanglement found: (0.9883998311890938, 0.3939867684050529)
succeeding entanglement gen at left at time: 1000400005000
succeeding entanglement gen at right at time: 1000400005000.0
succeeding entanglement gen at left at time: 1000500010000
succeeding entanglement gen at left at time: 1000500010000
succeeding entanglement gen at right at time: 1000500010000.0
succeeding entanglement gen at right

KeyboardInterrupt: 